This workflow focuses on topic creation rather than conversation creation. Good idea to run this (and anything involving loading >=7B models) in Lambda Labs? (I have 8xA100 in California) though be careful to always terminate the Lambda Labs instance after because its costs run from the time it's launched to the time it's properly terminated, not just when it runs.

In [ ]:
#from 1b personas demo advice
!pip install datasets transformers vllm

In [ ]:
#this is necessary to load "intermediate" datasets like the animal welfare topics dataset
#from google.colab import drive
#drive.mount('/content/drive')

This is the script executing experiments. Please change the drive mounting process if you see your own account's google folders labeled differently.

In [ ]:
#%cd /content/drive/MyDrive/ProjectEmpathy/Codebase/persona-hub-main/

In [ ]:
#%ls

Note: I tried 0.5b model below but it hallucinated something to the effect of "Pork is suitable for vegetarians and vegans." 🤡

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/Open-Paws/Datasets/refs/heads/main/animal_species/FAO_Domesticated_Animal_breeds.csv'
domesticDF = pd.read_csv(url)

In [ ]:
#!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset('Kuppuram/animals')

# Access the training split
allAnimalsDF = dataset['train'].to_pandas()

# Now you can work with train_df as a Pandas DataFrame
print(allAnimalsDF.head())

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

animals.csv:   0%|          | 0.00/40.4k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2691 [00:00<?, ? examples/s]

                  animals
0                Aardvark
1                Aardwolf
2              Abyssinian
3   Abyssinian Guinea Pig
4      Acadian Flycatcher


In [ ]:
debateAVegandataset = load_dataset('sparrow8i8/DebateAVegan')

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

selftext.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/560 [00:00<?, ? examples/s]

In [ ]:
debateAVegandataset=debateAVegandataset['train'].to_pandas()

In [ ]:
debateAVegandataset['text'][0]

"I came here just for a second opinion on what I think on veganism.\nI just think that it's unhealthy to not eat anything from animals or plants.\nYou need to take nutrients from other things to survive.\nPlus,I think the increased meat production can feed a lot of people who don't get enough food,and it would be bad to decrease production."

In [ ]:
allAnimalsDF['animals'][0]

'Aardvark'

In [ ]:
domesticDF

array(['Ass', 'Cattle', 'Chicken', 'Dromedary', 'Goat', 'Horse', 'Sheep',
       'Yak (domestic)', 'Pig', 'Buffalo', 'Turkey', 'Guanaco', 'Llama',
       'Vicuña', 'Rabbit', 'Deer', 'Duck (domestic)', 'Goose (domestic)',
       'Guinea fowl', 'Muscovy duck', 'Peacock', 'Pigeon', 'Alpaca',
       'American Bison', 'Cassowary', 'Dog', 'Dromedary Bactrian Camel',
       'Emu', 'Ostrich', 'Bactrian camel', 'Partridge', 'Pheasant',
       'Quail', 'Duck(domestic) Muscovy duck', 'Guinea pig', 'Nandu',
       'Swallow', 'Gaur', 'Mithun'], dtype=object)

In [ ]:
from vllm import LLM, SamplingParams

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
#convo templates

lines=""
#2.5 instead of 2, below
topicllm = LLM(model="Qwen/Qwen2.5-7B-Instruct", cpu_offload_gb=2)


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

INFO 10-02 00:45:19 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='Qwen/Qwen2.5-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Qwen/Qwen2.5-7B-Instruct, use_v2_block_manager=False, num_scheduler_steps=1, multi_step_stream_outputs=Fa

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 10-02 00:46:06 model_runner.py:1025] Loading model weights took 12.1953 GB
INFO 10-02 00:46:09 gpu_executor.py:122] # GPU blocks: 21121, # CPU blocks: 4681
INFO 10-02 00:46:12 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-02 00:46:12 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-02 00:46:48 model_runner.py:1456] Graph capturing finished in 36 secs.


In [ ]:
sampling_params = SamplingParams(temperature=0.7, max_tokens=2000)
input_list = []
domestic = []
ind_animals = []
all_animals= []
reddit_prompts= []

for topic in reddit_prompts:
    reddit_prompts.append(f"Generate a hundred topics related to animal welfare by using the following text as inspiration:{topic}"),

for domesticAnimal in domesticDF:
    domestic.append(f"Generate a hundred topics related to farmed animal suffering with respect to {domesticAnimal}. Format the topics as a numbered list"),
    domestic.append(f"Generate a hundred topics related to how we can effectively reduce suffering for all dometicated {domesticAnimal}. Format the topics as a numbered list")
    domestic.append(f"Generate a hundred topics related to {domesticAnimal} and their exploitation by humans. Format the topics as a numbered list"),

for animal in allAnimalsDF:

    ind_animals.append(f"Generate a hundred topics related to {animal} and their welfare as faced by humans today. Format the topics as a numbered list"),
    ind_animals.append(f"Generate a hundred topics related to {animal} and their welfare that will be faced by future humans. Format the topics as a numbered list"),
    ind_animals.append(f"Generate a hundred topics related to {animal} and their exploitation by humans. Format the topics as a numbered list"),
    ind_animals.append(f"Generate a hundred topics related to {animal} suffering. Format the topics as a numbered list"),
    ind_animals.append(f"Generate a hundred topics related to the study of morality of {animal} suffering and sentience. Format the topics as a numbered list"),
    ind_animals.append(f"Generate a hundred topics related to {animal} welfare and society's perception of it. Format the topics as a numbered list"),
    ind_animals.append(f"Generate a hundred topics related to perceptions of {animal} and the need for them around the globe. Format the topics as a numbered list"),
    ind_animals.append(f"Generate a hundred topics related to how we can improve conservation efforts for {animal}. Format the topics as a numbered list")

all_animals.append(f"Generate a hundred topics related to farmed animal suffering with respect to animals. Format the topics as a numbered list")
all_animals.append(f"Generate a hundred topics related to how we can effectively reduce suffering for all dometicated animals. Format the topics as a numbered list")
all_animals.append(f"Generate a hundred topics related to animals and their exploitation by humans. Format the topics as a numbered list")
all_animals.append(f"Generate a hundred topics related to animals and their welfare as faced by humans today. Format the topics as a numbered list"),
all_animals.append(f"Generate a hundred topics related to animals and their welfare that will be faced by future humans. Format the topics as a numbered list"),
all_animals.append(f"Generate a hundred topics related to animals and their exploitation by humans. Format the topics as a numbered list"),
all_animals.append(f"Generate a hundred topics related to animal suffering. Format the topics as a numbered list"),
all_animals.append(f"Generate a hundred topics related to the study of morality of animal suffering and sentience. Format the topics as a numbered list"),
all_animals.append(f"Generate a hundred topics related to animal welfare and society's perception of it. Format the topics as a numbered list"),
all_animals.append(f"Generate a hundred topics related to perceptions of animals and the need for them around the globe. Format the topics as a numbered list"),
all_animals.append(f"Generate a hundred topics related to how we can improve conservation efforts for animals. Format the topics as a numbered list")

input_list = [domestic, ind_animals, all_animals, reddit_prompts]
#for prompt in input_list:
#    prompt += ". Start each topic on a new line and start them without any numbers."
response_text=topicllm.generate(input_list,sampling_params=sampling_params,
                    use_tqdm=False)
#response_text

In [ ]:
#a sample of what we need
for i in range(2,7):
    print(response_text[i].outputs[0].text.strip())

1. Animal testing and its ethical implications
2. The use of animals in entertainment, such as circuses and rodeos
3. Factory farming and its impact on animal welfare
4. The role of animals in sports and their exploitation
5. Animal fighting and its illegal nature
6. The use of animals in scientific research
7. The ethics of using animals for fashion, such as fur and leather
8. The commercial exploitation of farm animals for meat and dairy
9. The exploitation of marine animals for aquariums and swim-with-dolphins programs
10. The use of animals in military and defense
11. The exploitation of exotic animals as pets
12. The ethics of using animals for hunting and sport
13. The role of animals in religious and cultural practices
14. The exploitation of animals in the food industry
15. The use of animals in wildlife tourism
16. The exploitation of animals in the cosmetic industry
17. The ethics of using animals in drug testing
18. The use of animals in animal agriculture
19. The exploitati

In [ ]:
#cleaner AI
# Put dialogue into chat template
!pip install Jinja2==3.1.0
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
clean=""
for i in range(len(response_text)):
  text = response_text[i].outputs[0].text
  prompt = [
      {"role": "system", "content": "Please return a list of cleaned topics by removing topics that do not make sense, are duplicates of other topics or are unrelated to animal welfare. Please return no extra text or words besides the cleaned lists.\n\n"}
      ,
      {"role": "user", "content": text}]
  prompts = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
  cleaned_text=topicllm.generate(prompts,sampling_params=sampling_params,use_tqdm=False)
  cleaned_text = cleaned_text[0].outputs[0].text
  print(cleaned_text)
  clean+=cleaned_text

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 3.5 MB/s eta 0:00:00a 0:00:01
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


ImportError: apply_chat_template requires jinja2>=3.1.0 to be installed. Your version is 3.0.3.

In [ ]:
#writing it to files
for i in range(len(input_list)):
    topic = input_list[i]
    text = response_text[i].outputs[0].text
    #https://www.pythontutorial.net/python-basics/python-write-text-file/
    with open(f"{topic}.txt", "w") as f:
        f.write(text)
#and lots of new text files should pop up!

In [ ]:
#when out of memory OR ABOUT TO CHANGE MODEL TYPES
# Clean up GPU memory use of the model
#del llm
#import gc
#gc.collect()